# Introduction to Browser Automation with Playwright in Python

This lesson covers automating web browser interactions using the Playwright library in Python, drawing from Python programming fundamentals.

## Basic Setup and Imports

**<span style="color: red">DISCLAIMER: this notebook will only run locally (not on Google Colab)</span>**

%pip install playwright
%playwright install

In [ ]:
from playwright.sync_api import Playwright, sync_playwright, expect
import os
import sys

## Understanding Browser Automation Flow

In [ ]:
def search_movies(playwright: Playwright, movie_title: str) -> None:
    # Launch browser in visible mode
    browser = playwright.chromium.launch(headless=False)
    context = browser.new_context()
    
    # Create new page
    page = context.new_page()
    
    try:
        # Navigate and perform actions
        page.goto("https://imdb.com")
        search_box = page.get_by_label("Search IMDb")
        search_box.fill(movie_title)
        search_box.press("Enter")
        
        # Click first movie result
        page.get_by_role("link", name=movie_title, exact=True).first.click()
        
        # Add to watchlist if logged in
        page.get_by_role("button", name="Add to Watchlist").click()
        
    finally:
        # Clean up
        context.close()
        browser.close()

# Usage
with sync_playwright() as playwright:
    movie = sys.argv[1] if len(sys.argv) > 1 else "The Matrix"
    search_movies(playwright, movie)

## Using Environment Variables

In [ ]:
def get_credentials():
    username = os.environ.get("MOVIE_DB_USER")
    password = os.environ.get("MOVIE_DB_PWD")
    
    if not username or not password:
        raise ValueError("Missing credentials in environment variables")
        
    return username, password

## Advanced Page Interactions

In [ ]:
def login_to_site(page, username: str, password: str):
    page.get_by_role("link", name="Sign in").click()
    
    # Fill login form
    page.get_by_label("Username").fill(username)
    page.get_by_label("Password").fill(password)
    
    # Submit form
    page.get_by_role("button", name="Sign in").click()
    
    # Wait for navigation
    page.wait_for_load_state("networkidle")

## Error Handling and Retries

In [ ]:
from typing import Callable
import time

def retry_action(action: Callable, max_attempts: int = 3, delay: float = 1.0):
    """Retry an action with exponential backoff"""
    for attempt in range(max_attempts):
        try:
            return action()
        except Exception as e:
            if attempt == max_attempts - 1:
                raise
            time.sleep(delay * (2 ** attempt))

## Complete Example: Movie Search and Rating

In [ ]:
def rate_movie(playwright: Playwright, movie_title: str, rating: int) -> None:
    browser = playwright.chromium.launch(headless=False)
    context = browser.new_context()
    page = context.new_page()
    
    try:
        # Setup
        page.goto("https://imdb.com")
        username, password = get_credentials()
        
        # Login
        retry_action(lambda: login_to_site(page, username, password))
        
        # Search and rate
        search_box = page.get_by_label("Search IMDb")
        search_box.fill(movie_title)
        search_box.press("Enter")
        
        # Find and click movie
        movie_link = page.get_by_role("link", name=movie_title, exact=True).first
        movie_link.click()
        
        # Rate movie
        rating_button = page.get_by_role("button", name=f"Rate {rating}")
        rating_button.click()
        
    finally:
        context.close()
        browser.close()

# Example usage
if __name__ == "__main__":
    with sync_playwright() as playwright:
        movie = sys.argv[1] if len(sys.argv) > 1 else "Inception"
        rating = int(sys.argv[2]) if len(sys.argv) > 2 else 5
        rate_movie(playwright, movie, rating)

This code demonstrates core Playwright concepts:
- Browser and context management
- Page navigation and interaction
- Form filling and button clicking
- Error handling and retries
- Environment variable usage
- Command line arguments
- Type hints and documentation

The example uses IMDb instead of Letterboxd but follows similar patterns while introducing additional programming concepts.